## Math 157: Intro to Mathematical Software
## UC San Diego, winter 2018

## Final project, part 1: due March 16, 2018

The final project consists of two parts, each of equal value. Both parts will be submitted in this folder, just like a homework assignment. However, the final project is *not* equivalent to a homework assignment from the point of view of course grading; it is a separate contribution, and cannot be dropped.

Part 1 is a problem set in the style of the preceding homework assignments, but with material covering the whole course.
It consists of 8 problems, each of equal value. As usual, please enter all answers within this notebook except as specified, and cite all sources and collaborators.

Throughout this problem set, use the SageMath 8.1 kernel except as specified.

### Problem 1: Symbolic sums

Grading criteria: correctness of code.

Demonstrate that Sage is able to evaluate the following infinite sums (without manually encoding known mathematical formulas).

1. Compute $\sum_{n=1}^{\infty} \frac{(-1)^n}{n}$ symbolically.
1. Compute $\sum_{n=1}^{\infty} \frac{1}{n^2}$ symbolilcally.
1. Compute $\sum_{n=1}^{\infty} \frac{1}{n^3}$ both symbolically (in terms of the Riemann Zeta function) and numerically.
1. Compute $\sum_{n=1}^{\infty} \frac{1}{n^4}$ symbolically.

In [4]:
n = var('n')
sum((-1)**n/n, n, 1, oo)

-log(2)

In [5]:
sum(1/n**2, n, 1, oo)

1/6*pi^2

In [6]:
sum(1/n**3, n, 1, oo), N(zeta(3))

(zeta(3), 1.20205690315959)

In [4]:
sum(1/n**4, n, 1, oo)

1/90*pi^4

### Problem 2: Determinants

Grading criteria: correctness of code and explanations.

2a. Implement a function called `my_det` that calculates the determinant of a matrix recursively using expansion by cofactors. The Matrix methods `delete_columns` and `delete_rows` may come in handy. You can assume that your input is a square matrix.

In [5]:
def my_det(M):
    if M.ncols() == 1 and M.nrows() == 1:
        return M[0,0]
    else:
        sum = 0
        for j in range(M.ncols()):
            cof = (-1)^(0+j) * M[0,j] #expanding along 1st row
            M_sub = M.delete_rows([0])
            M_sub = M_sub.delete_columns([j])
            sum += cof * my_det(M_sub)
    return sum

2b. Check that your function agrees with Sage's built-in function on a random 5 by 5 matrix over $\mathbb{Z}$.

In [6]:
A = random_matrix(ZZ,5,5)
A.determinant() == my_det(A)

True

2c. Do some timings for random $n \times n$ matrices over $\mathbb{Z}$ for $n=5, \dots, 10$ comparing your function and Sage's built-in function. Make a single plot containing both sets of results.

In [7]:
time_built_in = [0 for x in range(5,11)]
time_my_det = [0 for x in range(5,11)]
import time

for n in range(5,11):
    A = random_matrix(ZZ,n,n)
    #built in
    t = time.time()
    A.determinant()
    t = time.time() - t
    time_built_in[n-5] = t

    #my function
    s = time.time()
    my_det(A)
    s = time.time() - s
    time_my_det[n-5] = s

In [13]:
print(time_built_in)
print(time_my_det)

import matplotlib.pyplot as plt
plt.plot(range(5,11),time_built_in,'b',linewidth=3)
plt.plot(range(5,11),time_my_det,'r')
plt.show()
#the scale of built in function times is so much smaller than scale of my fucntion times

[2.193450927734375e-05, 2.5987625122070312e-05, 2.9087066650390625e-05, 3.314018249511719e-05, 4.38690185546875e-05, 4.315376281738281e-05]
[0.016534090042114258, 0.0895700454711914, 0.5283520221710205, 3.5017831325531006, 33.60978317260742, 295.2963559627533]


2d. Explain the results of 2c in terms of the asymptotic complexity of the two methods.

my_det requires a lot of recursive function calls which exponentially increases the complexity of the computation as n increases. The built in determinant funciton is very efficient and does not see this exponential in run time for the sample n.

### Problem 3: Back to the poker table

Grading criteria: correctness of code.

In a previous exercise, you constructed a "deck" of 52 playing cards consisting of the Cartesian product of "ranks" and "suits",
and wrote a function to classify a set of 5 (distinct) cards as a poker hand. You may wish to reuse your answer to that problem here.

3a. Write a function which, given two poker hands `h1` and `h2`, returns 1 if `h1` is the higher-ranking hand, -1 if `h2` is the higher-ranking hand, and 0 if the two hands are equally ranked. Note that poker hands are primarily sorted by their type, but there are rules to break some ties even among hands of the same type. (For example, a pair of queens outranks a pair of jacks.)

In [46]:
### reusing code from previous hw
#suits = Set(['C','D','H','S'])
#ranks = Set(["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K","A"]) replacing suits with 1,2,3,4 and face cards with 11,12,13,14
suits = Set([1,2,3,4])
ranks = Set([2,3,4,5,6,7,8,9,10,11,12,13,14])
deck = cartesian_product([suits, ranks])
deck.cardinality()

Hands = Subsets(deck, 5)
Hands.cardinality() #matches binomial(52,5)
iterator_hands = iter(Hands)

In [47]:
#Helper Functions
#This function counts how many of each rank there are in the hand
#https://stackoverflow.com/questions/2161752/how-to-count-the-frequency-of-the-elements-in-a-list
def rank_counts(hand):
    from itertools import groupby
    rank_seen = sorted([b for (a,b) in hand])
    rank_seen_count = [len(list(group)) for key, group in groupby(rank_seen)]
    rank_seen = sorted(list(set(rank_seen)))
    return (rank_seen,rank_seen_count)


def is_royal_flush(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight AND all same suit
    if (rank_seen[0]==10) and (rank_seen[1]==11) and (rank_seen[2]==12) and (rank_seen[3]==13) and (rank_seen[4]==14) and len(set(a for (a,b) in hand)) == 1:
        return True
    return False


def is_straight_flush(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight AND all same suit
    if ((rank_seen[0]==2) and (rank_seen[1]==3) and (rank_seen[2]==4) and (rank_seen[3]==5) and (rank_seen[4]==14) or (rank_seen[0]+1==rank_seen[1]) and (rank_seen[1]+1==rank_seen[2]) and (rank_seen[2]+1==rank_seen[3]) and (rank_seen[3]+1==rank_seen[4])) and len(set(a for (a,b) in hand)) == 1 and (not is_royal_flush(hand)):
        return True
    return False


def is_four_of(hand):
    P = rank_counts(hand)
    #check if hand has 2 types of ranks
    if len(P[0]) != 2:
        return False
    if (1 in P[1]) and (4 in P[1]):
        return True
    else:
        return False

def is_full_house(hand):
    P = rank_counts(hand)
    #check if hand has 2 types of ranks
    if len(P[0]) != 2:
        return False
    if (2 in P[1]) and (3 in P[1]):
        return True
    else:
        return False

#https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/tutorial.html
def is_flush(hand):
    if (len(set(a for (a,b) in hand))==1) and (not is_straight_flush(hand)) and (not is_royal_flush(hand)):
        return True
    return False
#a is the suit of the card, b is the rank. Set of the suits of the cards only shows distinct suits i.e. if all suits are the same, should have length 1


def is_straight(hand):
    rank_seen = sorted([b for (a,b) in hand])
    #if the n+1 rank is 1 larger than the nth rank for all 0<=n<=3, then it is a straight
    if ((rank_seen[0]==2) and (rank_seen[1]==3) and (rank_seen[2]==4) and (rank_seen[3]==5) and (rank_seen[4]==14) or (rank_seen[0]+1==rank_seen[1]) and (rank_seen[1]+1==rank_seen[2]) and (rank_seen[2]+1==rank_seen[3]) and (rank_seen[3]+1==rank_seen[4])) and (not is_straight_flush(hand)) and (not is_royal_flush(hand)):
        return True
    return False


def is_three_of(hand):
    P = rank_counts(hand)
    #check if hand has 3 types of ranks
    if len(P[0]) != 3:
        return False
    if 3 in P[1]:
        return True
    else:
        return False


def is_two_pair(hand):
    P = rank_counts(hand)
    #check if hand has 3 types of ranks
    if len(P[0]) != 3:
        return False
    if 2 in P[1]:
        return True
    else:
        return False


def is_one_pair(hand):
    P = rank_counts(hand)
    #check if hand has 4 types of ranks
    if len(P[0]) != 4:
        return False
    if 2 in P[1]:
        return True
    else:
        return False


def is_high_card(hand):
    P = rank_counts(hand)
    #check if hand has 5 types of ranks
    if len(P[0]) != 5 or is_straight(hand) or is_flush(hand) or is_straight_flush(hand) or is_royal_flush(hand):
        return False
    else:
        return True

#return the class of the hand as a numeric value with royal straight flush as the highest value (9) and high card as the lowest value (0)
def class_hand(hand):
    if is_high_card(hand):
        return 0
    elif is_one_pair(hand):
        return 1
    elif is_two_pair(hand):
        return 2
    elif is_three_of(hand):
        return 3
    elif is_straight(hand):
        return 4
    elif is_flush(hand):
        return 5
    elif is_full_house(hand):
        return 6
    elif is_four_of(hand):
        return 7
    elif is_straight_flush(hand):
        return 8
    else:
        return 9

In [48]:
#assuming hands are equal value: returns 1 if value of hand 1 is bigger than value of hand 2, 2 if value of hand 1 is smaller than value of hand 2, 0 is they are equal value
def high_value(h1,h2):
    (A,B) = rank_counts(h1)
    (P,Q) = rank_counts(h2)
    #print(A,B,P,Q)

    if len(B)==5: #straight flush, straight, flush, high card
        for j in [4,3,2,1,0]:
            if A[j] > P[j]:
                return 1
            elif A[j] < P[j]:
                return -1
            elif A[j] == P[j] and j==0:
                return 0
    elif len(B)==2: #4 of a kind, full house
        index1 = [i for i in range(len(B)) if B[i]==max(B)]
        index2 = [i for i in range(len(Q)) if Q[i]==max(Q)]
        if A[index1[0]] > P[index2[0]]:
            return 1
        elif A[index1[0]] < P[index2[0]]:
            return -1
        elif A[index1[0]] == P[index2[0]]:
            if is_four_of(h1):
                k = 3
            elif is_full_house(h1):
                k = 1
            index1 = [i for i in range(len(B)) if B[i]==max(B)-k]
            index2 = [i for i in range(len(Q)) if Q[i]==max(Q)-k]
            if A[index1[0]] > P[index2[0]]:
                return 1
            elif A[index1[0]] < P[index2[0]]:
                return -1
            elif A[index1[0]] == P[index2[0]]:
                return 0
    elif is_three_of(h1): #3 of a kind
        index1 = [i for i in range(len(B)) if B[i]==max(B)]
        index2 = [i for i in range(len(Q)) if Q[i]==max(Q)]
        if A[index1[0]] > P[index2[0]]:
            return 1
        elif A[index1[0]] < P[index2[0]]:
            return -1
        else:
            index1 = [i for i in range(len(B)) if B[i]==max(B)-2] #returns 2 indicies of the 2 single ranks
            index2 = [i for i in range(len(Q)) if Q[i]==max(Q)-2]
            for j in [1,0]:
                if A[index1[j]] > P[index2[j]]:
                    return 1
                elif A[index1[j]] < P[index2[j]]:
                    return -1
                elif A[index1[j]] == P[index2[j]] and j==0:
                    return 0
    elif is_two_pair(h1): #2 pair
        index1 = [i for i in range(len(B)) if B[i]==max(B)] #returns 2 indicies of the 2 pair
        index2 = [i for i in range(len(Q)) if Q[i]==max(Q)]
        for j in [1,0]:
            if A[index1[j]] > P[index2[j]]:
                return 1
            elif A[index1[j]] < P[index2[j]]:
                return -1
            elif A[index1[j]] == P[index2[j]]:
                continue

        index1 = [i for i in range(len(B)) if B[i]==max(B)-1]
        index2 = [i for i in range(len(Q)) if Q[i]==max(Q)-1]
        if A[index1[0]] > P[index2[0]]:
            return 1
        elif A[index1[0]] < P[index2[0]]:
            return -1
        elif A[index1[0]] == P[index2[0]]:
            return 0
    elif is_one_pair(h1): #1 pair
        index1 = [i for i in range(len(B)) if B[i]==max(B)]
        index2 = [i for i in range(len(Q)) if Q[i]==max(Q)]
        if A[index1[0]] > P[index2[0]]:
            return 1
        elif A[index1[0]] < P[index2[0]]:
            return -1
        else:
            index1 = [i for i in range(len(B)) if B[i]==max(B)-1] #returns 3 indicies of the 3 single ranks
            index2 = [i for i in range(len(Q)) if Q[i]==max(Q)-1]
            for j in [2,1,0]:
                if A[index1[j]] > P[index2[j]]:
                    return 1
                elif A[index1[j]] < P[index2[j]]:
                    return -1
                elif A[index1[j]] == P[index2[j]] and j==0:
                    return 0

In [50]:
# #testing cases
# hand5 = {(2,9),(1,2),(3,5),(4,11),(2,11)} #2 pair case
# hand6 = {(2,9),(1,8),(3,5),(4,11),(2,11)}

# hand3 = {(2,9),(1,9),(3,9),(4,13),(2,13)} #full house case
# hand4 = {(2,9),(1,9),(3,9),(4,11),(2,11)}

# hand1 = {(2,4),(1,4),(3,4),(4,2),(2,8)} #3 of a kind case
# hand2 = {(2,4),(1,4),(3,4),(4,3),(2,10)}

# hand7 = {(2,5),(1,5),(3,8),(4,8),(2,11)} #2 pair
# hand8 = {(2,4),(1,4),(3,8),(4,8),(2,10)}

# handA = {(2,5),(1,5),(3,3),(4,9),(2,11)} #1 pair
# handB = {(2,5),(1,5),(3,7),(4,9),(2,11)}
# u = rank_counts(handA)
# type(u[1])

# best = {(2, 7), (2, 8), (1, 8), (3, 8), (4, 8)}
# i = {(3, 8), (2, 8), (1, 8), (1, 6), (4, 8)}

# high_value(best,i)

In [51]:
def higher_rank(h1,h2):
    if class_hand(h1) > class_hand(h2):
        return 1
    elif class_hand(h1) < class_hand(h2):
        return -1
    elif class_hand(h1) == class_hand(h2) and high_value(h1,h2)==1:
        return 1
    elif class_hand(h1) == class_hand(h2) and high_value(h1,h2)==-1:
        return -1
    else:
        return 0

In [52]:
# #testing cases
# for i in range(5):
#     handA = Hands.random_element()
#     handB = Hands.random_element()
#     print(handA)
#     print(handB)
#     print(higher_rank(handA,handB))

3b. Define a Python class called "Hand" with the following property: if `s1` and `s2` are two poker hands, then
```
h1 = Hand(s1)
h2 = Hand(s2)
print(h1 < h2)
```
will print True if and only if `s2` is (strictly) higher in ranking than `s1`. Your class should implement the method ``__lt__`` to achieve operator overloading. (Do not implement ``__cmp__``; this works in Python 2 but will fail in Python 3.)

In [53]:
class Hand:
    # rest of the definition goes here
    def __init__(self,hand):
        self.h = hand

    def __lt__(self,hand2):
        if higher_rank(self.h,hand2.h) == -1:
            return True
        else:
            return False

In [54]:
# Assume that s1 and s2 are two lists of five distinct cards each.
s1 = {(2,5),(1,5),(3,3),(4,9),(2,11)}
s2 = {(2,5),(1,5),(3,7),(4,9),(2,11)} #larger
h1 = Hand(s1)
h2 = Hand(s2)
print(h1 < h2)

True


### Problem 4: Texas Hold 'Em

Grading criteria: correctness of code, to be judged as if your solution to problem 3 is completely correct.

Texas Hold 'Em is the variety of poker most often shown on television. Each player is initially dealt two cards (kept hidden), and eventually five more cards are dealt (face up) on the table. Each player's holding is ranked according to the best hand that can be made from their two cards plus the five common cards. (The bidding aspect of the game is not relevant for this problem.)


3a. Write a function which, given a set of seven distinct cards, identifies the highest-ranking hand which can be made using some five of the cards. In case of a tie, you may return any hand which ties for the highest ranking.

In [55]:
#assumes set of 7 distinct cards has inputs of the form {(s,r),...,(s,r)} where suits range from 1,2,3,4 and ranks range from 2,...,14
def best_texas(cards):
    hand = Subsets(cards,5)
    best = hand.an_element() #initalize the best hand variable
#    print("initialized is",best)
#    count=0
    for i in iter(hand):
#        count+=1
#        print("-----------------------------------------------------------------")
#        print("best is",best)
#        print("i is",i)
        if higher_rank(best,i) == -1:
            best = i #if the rank of i is higher than the current best, replace it
        else:
            continue #if the rank of i is less than or equal to the current best, continue
#    print(best,count)
    return best

In [56]:
# #more testing
# suits = Set([1,2,3,4])
# ranks = Set([2,3,4,5,6,7,8,9,10,11,12,13,14])
# deck = cartesian_product([suits, ranks])
# deck.cardinality()

# cards = Subsets(deck, 7)
# c = {(3, 8), (2, 7), (1, 8), (1, 6), (4, 8), (1, 10), (2, 8)} #test cards
# print(c,type(c))
# # print("c is",c)
# best_texas(c)

3b. Write a function which, given two pairs of cards, computes the probability that for a random set of five common cards (where all nine cards in question are distinct), the first pair will outrank the second pair. For full credit, you should do this by exhausting over all possibilities for the common cards, rather than doing a random sample.

In [57]:
def best_texas_2pair(pair1,pair2,BOOT):
    #ensuring the rest of the remaining deck cards are distinct
    suits = Set([1,2,3,4])
    ranks = Set([2,3,4,5,6,7,8,9,10,11,12,13,14])
    deck = cartesian_product([suits, ranks])
    deck = set(deck) - pair1 - pair2 #remaining deck is 52-4=48 cards
    random_hand = Subsets(deck, 5)

    count = 0
    for i in range(BOOT):
        r = set(random_hand.random_element()) #select 5 random cards from the remaining deck
        hand1 = pair1.union(r); hand2 = pair2.union(r) #union the pair with 5 selected cards for a total of 7
        best1 = best_texas(hand1)
        best2 = best_texas(hand2)
        if Hand(best2) < Hand(best1):
            count+=1

    return count/BOOT

In [58]:
p1 = {(2,8),(4,8)}
p2 = {(4,7),(1,3)}
prob = best_texas_2pair(p1,p2,10**4)
print(prob,N(prob))

(8709/10000, 0.870900000000000)


3c. Write a function, given *one* pair of distinct cards, computes the probability that if one chooses another pair and five common cards at random from the remaining cards (so again all nine cards in question are distinct), the given pair will outrank the other pair. For this problem, instead of exhaustion, use $10^6$ random samples.

In [61]:
def best_texas_1pair(pair1,BOOT):
    #ensuring the rest of the remaining deck cards are distinct
    suits = Set([1,2,3,4])
    ranks = Set([2,3,4,5,6,7,8,9,10,11,12,13,14])
    deck = cartesian_product([suits, ranks])
    deckA = set(deck) - pair1 #remaining deck is 52-2=50 cards

    count = 0
    for i in range(BOOT):
        pair2 = set(Subsets(deckA,2).random_element()) #select a random pair from the deck
        deck = deckA - pair2 #remaining deck is 50-2=48 cards
        random_hand = Subsets(deck,5)
        r = set(random_hand.random_element()) #select 5 random cards from the remaining deck
        hand1 = pair1.union(r); hand2 = pair2.union(r) #union the pair with 5 selected cards for a total of 7
        best1 = best_texas(hand1)
        best2 = best_texas(hand2)
        if Hand(best2) < Hand(best1):
            count+=1

    return count/BOOT

In [62]:
p1 = {(2,8),(4,8)}
prob = best_texas_1pair(p1,10**4)
print(prob,N(prob))

(1369/2000, 0.684500000000000)


3d. Determine the values of the function you wrote in 3c for all possible pairs of distinct tards. Note that this probability should depend only on the values of the two cards and (if the values are distinct) whether the two cards are of the same or different suits.

In [64]:
dict = {}
different_suit = [(i,j) for i in range(2,14+1) for j in range(i,14+1)]
same_suit = [(i,j) for i in range(2,14+1) for j in range(i,14+1) if i != j]
BootNumber = 10**3

for i in range(len(different_suit)):
    pair = different_suit[i]
    name = '(D,%d,%d)' % (pair[0],pair[1])
    pair = {(1,pair[0]),(2,pair[1])} #suits are 1&2 which are placeholders to represent different suits
    prob = best_texas_1pair(pair,BootNumber)
    temp = {name: prob}
    dict.update(temp)

for i in range(len(same_suit)):
    pair = same_suit[i]
    name = '(S,%d,%d)' % (pair[0],pair[1])
    pair = {(1,pair[0]),(1,pair[1])} #suits are 1&1 which are placeholders to represent same suits
    prob = best_texas_1pair(pair,BootNumber)
    temp = {name: prob}
    dict.update(temp)

In [66]:
dict

{'(D,10,10)': 383/500,
 '(D,10,11)': 529/1000,
 '(D,10,12)': 549/1000,
 '(D,10,13)': 573/1000,
 '(D,10,14)': 313/500,
 '(D,11,11)': 97/125,
 '(D,11,12)': 567/1000,
 '(D,11,13)': 597/1000,
 '(D,11,14)': 83/125,
 '(D,12,12)': 399/500,
 '(D,12,13)': 609/1000,
 '(D,12,14)': 317/500,
 '(D,13,13)': 413/500,
 '(D,13,14)': 83/125,
 '(D,14,14)': 173/200,
 '(D,2,10)': 411/1000,
 '(D,2,11)': 23/50,
 '(D,2,12)': 219/500,
 '(D,2,13)': 247/500,
 '(D,2,14)': 53/100,
 '(D,2,2)': 521/1000,
 '(D,2,3)': 309/1000,
 '(D,2,4)': 137/500,
 '(D,2,5)': 149/500,
 '(D,2,6)': 8/25,
 '(D,2,7)': 339/1000,
 '(D,2,8)': 43/125,
 '(D,2,9)': 179/500,
 '(D,3,10)': 413/1000,
 '(D,3,11)': 453/1000,
 '(D,3,12)': 93/200,
 '(D,3,13)': 119/250,
 '(D,3,14)': 569/1000,
 '(D,3,3)': 107/200,
 '(D,3,4)': 301/1000,
 '(D,3,5)': 8/25,
 '(D,3,6)': 43/125,
 '(D,3,7)': 373/1000,
 '(D,3,8)': 337/1000,
 '(D,3,9)': 367/1000,
 '(D,4,10)': 83/200,
 '(D,4,11)': 431/1000,
 '(D,4,12)': 23/50,
 '(D,4,13)': 13/25,
 '(D,4,14)': 563/1000,
 '(D,4,4)':

### Problem 5: Adjacent states

Grading criteria: correctness of code.

Throughout this problem, by "US states" we mean the 50 US states *plus* the District of Columbia.

5a. The following string, adapted from [this source](https://writeonly.wordpress.com/2009/03/20/adjacency-list-of-states-of-the-united-states-us/), represents the adjacencies among US states, as represented by their [standard postal abbreviations](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

Starting from `s`, construct a graph whose vertices are labeled by the postal abbreviations, with an edge between two vertices if and only if they correspond to adjacent states.

In [24]:
s = """
AK
AL,MS,TN,GA,FL
AR,MO,TN,MS,LA,TX,OK
AZ,CA,NV,UT,NM
CA,OR,NV,AZ
CO,WY,NE,KS,OK,NM,UT
CT,NY,MA,RI
DC,MD,VA
DE,MD,PA,NJ
FL,AL,GA
GA,FL,AL,TN,NC,SC
HI
IA,MN,WI,IL,MO,NE,SD
ID,MT,WY,UT,NV,OR,WA
IL,IN,KY,MO,IA,WI
IN,MI,OH,KY,IL
KS,NE,MO,OK,CO
KY,IN,OH,WV,VA,TN,MO,IL
LA,TX,AR,MS
MA,RI,CT,NY,NH,VT
MD,VA,WV,PA,DC,DE
ME,NH
MI,WI,IN,OH
MN,WI,IA,SD,ND
MO,IA,IL,KY,TN,AR,OK,KS,NE
MS,LA,AR,TN,AL
MT,ND,SD,WY,ID
NC,VA,TN,GA,SC
ND,MN,SD,MT
NE,SD,IA,MO,KS,CO,WY
NH,VT,ME,MA
NJ,DE,PA,NY
NM,AZ,CO,OK,TX
NV,ID,UT,AZ,CA,OR
NY,NJ,PA,VT,MA,CT
OH,PA,WV,KY,IN,MI
OK,KS,MO,AR,TX,NM,CO
OR,CA,NV,ID,WA
PA,NY,NJ,DE,MD,WV,OH
RI,CT,MA
SC,GA,NC
SD,ND,MN,IA,NE,WY,MT
TN,KY,VA,NC,GA,AL,MS,AR,MO
TX,NM,OK,AR,LA
UT,ID,WY,CO,AZ,NV
VA,NC,TN,KY,WV,MD,DC
VT,NY,NH,MA
WA,ID,OR
WI,MI,MN,IA,IL
WV,OH,PA,MD,VA,KY
WY,MT,SD,NE,CO,UT,ID
"""

In [25]:
# M = matrix(51,51)
# row = s.splitlines() #reformatting string
# states = [[] for x in range(51)]
# for i in range(0,51):
#     if row[i+1] == "AK" or row[i+1] == "HI":
#         states[i] = row[i+1]
#     else:
#         states[i] = row[i+1].split(',')[0]

# for i in range(0,51): #indexing over the adjacent states #indexing over the 51 states
#     if row[i+1] == "AK" or row[i+1] == "HI":
#         adj = row[i+1]
#     else:
#         adj = row[i+1].split(',') #formatting

#     if adj == "AK" or adj == "HI":
#         continue #skips this row because alaska and hawaii are not connected to anything
#     else:
#         for j in range(0,51): 
#             for k in range(1,len(adj)):
#                 if adj[k] == states[j]:
#                     M[i,j] = 1

In [26]:
H = Graph()
row = s.splitlines() #reformatting string
states = [[] for x in range(51)]
for i in range(0,51):
    if row[i+1] == "AK" or row[i+1] == "HI":
        H.add_vertex(row[i+1])
    else:
        H.add_vertex(row[i+1].split(',')[0])

for i in range(0,51): #indexing over the adjacent states #indexing over the 51 states
    if row[i+1] == "AK" or row[i+1] == "HI":
        continue #no edges
    else:
        adj = row[i+1].split(',') #formatting

    if adj == "AK" or adj == "HI":
        continue #skips this row because alaska and hawaii are not connected to anything
    else:
        for j in range(1,len(adj)):
            H.add_edge(adj[0],adj[j])

In [27]:
H.plot()

5b. Compute a maximum flow and a minimum cut for this graph, using California as the source and the District of Columbia as the sink.

In [29]:
P = H.flow('CA','DC',value_only=False)
print(P[0])
P[1]

2


In [31]:
H.edge_cut('CA','DC')

2

### Problem 6: Cholesky dempositions

Grading criteria: correctness of explanations and code.

6a. In your words, explain what the *Cholesky decomposition* of a matrix is, for which matrices it is defined, and why it is useful in numerical linear algebra.

Cholesky decompostion is defined only on Hermitian positive-definite square nxn matricies where it decomposes the matrix A into a lower triangular matrix and an upper triangular matrix such that $A=L*U$ This is useful in numerical linear algebra because a generic system of linear equations $Ax=b$ can be rewritten as $LUx=b$. Let $Ux=y$ and solve for $Ly=b$ which can easily be solved for by forward substitution then solving $Ux=y$ using back substituion. Solving using forward and back substitution is cheaper than the generic approach to solving $Ax=b$.

6b. Use Sage's `cholesky` function to compute the Cholesky decomposition of the matrix
$$
M = \left(\begin{array}{rrrrr}
3 & -1 & -1 & -3 & -2 \\
-1 & 6 & 5 & 1 & 2 \\
-1 & 5 & 6 & 1 & 3 \\
-3 & 1 & 1 & 5 & 1 \\
-2 & 2 & 3 & 1 & 6
        \end{array}\right).
$$

In [33]:
M = matrix([[3,-1,-1,-3,-2],[-1,6,5,1,2],[-1,5,6,1,3],[-3,1,1,5,1],[-2,2,3,1,6]])
L = M.cholesky()
#L*L.transpose()
L

[  1.732050807568878?                    0                    0                    0                    0]
[-0.5773502691896258?   2.380476142847617?                    0                    0                    0]
[-0.5773502691896258?   1.960392117639214?   1.350381209794000?                    0                    0]
[ -1.732050807568878?                    0                    0   1.414213562373095?                    0]
[ -1.154700538379252?  0.5601120336112039?  0.9147743679249674? -0.7071067811865475?   1.736700616761008?]

6c. Repeat the computation using numpy.

In [34]:
import numpy as np
np.linalg.cholesky(M)

array([[  1.73205081e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -5.77350269e-01,   2.38047614e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -5.77350269e-01,   1.96039212e+00,   1.35038121e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -1.73205081e+00,  -9.32773914e-17,   0.00000000e+00,
          1.41421356e+00,   0.00000000e+00],
       [ -1.15470054e+00,   5.60112034e-01,   9.14774368e-01,
         -7.07106781e-01,   1.73670062e+00]])

6d. Repeat the computation using Octave. You may either switch to the Octave kernel or call Octave directly from Sage.

In [4]:
#Octave kernel
M = [3,-1,-1,-3,-2; -1,6,5,1,2; -1,5,6,1,3; -3,1,1,5,1; -2,2,3,1,6];
chol(M,'lower')

ans =


   1.73205   0.00000   0.00000   0.00000   0.00000


  -0.57735   2.38048   0.00000   0.00000   0.00000


  -0.57735   1.96039   1.35038   0.00000   0.00000


  -1.73205  -0.00000   0.00000   1.41421   0.00000


  -1.15470   0.56011   0.91477  -0.70711   1.73670


6e. Repeat the computation using Julia. You will need to switch to the Julia kernel for this.

In [12]:
#Julia kernel
M = [3 -1 -1 -3 -2; -1 6 5 1 2; -1 5 6 1 3; -3 1 1 5 1; -2 2 3 1 6]
C = chol(M)
transpose(C)

5×5 LowerTriangular{Float64,Array{Float64,2}}:
  1.73205    ⋅            ⋅          ⋅         ⋅    
 -0.57735   2.38048       ⋅          ⋅         ⋅    
 -0.57735   1.96039      1.35038     ⋅         ⋅    
 -1.73205  -9.32774e-17  0.0        1.41421    ⋅    
 -1.1547    0.560112     0.914774  -0.707107  1.7367

### Problem 7: Face completion

Grading criteria: correctness of code and answers.

**Special note**: in this problem, you will be running code in another notebook. However, you will only be credited for answers entered in this notebook.

Go to the [scikit-learn example of face completion](http://scikit-learn.org/stable/auto_examples/plot_multioutput_face_completion.html); download the script as a Jupyter notebook; upload it into this folder in your project; and run all cells using the Python 3 (Ubuntu Linux) kernel. Then answer the following questions.

7a. What fraction of the data was used for testing? And what code can be used to determine this?

25% of the data is being used for testing. The code snippet below was used to determine the testing size.

7b. Suppose we want to predict the top half of the face from the bottom. Which lines of code need to be changed, and to what?

7c. Suppose we want to use a [Multilayer perceptron regressor](https://en.wikipedia.org/wiki/Multilayer_perceptron) in addition to the four other estimators used in the sample. What lines of code need to be added, and where? (In each location where one or more lines is to be added, include one existing line before and after your addition.)

### Problem 8: Finding (discrete logarithms using) Nemo

Grading criteria: correctness of code.

For this problem, use the Julia kernel.

8a. Demonstrate the (usual, not elliptic) Diffie-Hellman key exchange in Julia using the prime `p=2171645417` and the primitive root `g=3`. That is, Alice and Bob choose random secrets and use them to establish a common key.

In [0]:
using Nemo

8b. Implement the baby-step-giant-step algorithm in Julia using the same `p` and `g`, including a test for correctness.